In [ ]:
import os
import re
import json
import ast
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
import ast
import pandas as pd


In [ ]:
main_path = '../gsm8k/LLMjudge/'
files = os.listdir(main_path)
len(files)

In [ ]:
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files)

In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
files[0]

In [ ]:
with open(main_path + files[0], 'r') as f:
        data = json.load(f)
data

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_emb(question_dict):
    encoded_input = tokenizer(question_dict, padding=True, truncation=False, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


In [ ]:
for j, file_name in enumerate(files):
    print(j)
    
    with open(main_path + file_name , 'r') as f:
        data = json.load(f)
        
    reason = data['reason']
    question = data['question']
    answer = data['answer']
    GT_answer = data['GT_answer']
    GT_reason = data['GT_reason']  
    generated_question = data['generated_question']
    evaluation = data['evaluation']
    
    question_dict = {}
    q1_start = re.search("question1",generated_question[0]).span()[0]
    q1_end = re.search("question1",generated_question[0]).span()[1]

    q2_start = re.search("question2",generated_question[1]).span()[0]
    q2_end = re.search("question2",generated_question[1]).span()[1]

    q3_start = re.search("question3",generated_question[2]).span()[0]
    q3_end = re.search("question3",generated_question[2]).span()[1]

    q1 = generated_question[0][q1_end + 4:]
    q1 = q1.replace('"', "")
    q2 = generated_question[1][q2_end + 4:]
    q2 = q2.replace('"', "")
    q3 = generated_question[2][q3_end + 4:]
    q3 = q3.replace('"', "")
    
    temp_ques = [question]
    temp_ques.append(q1)
    temp_ques.append(q2)
    temp_ques.append(q3)
    
    sentence_embeddings = get_emb(temp_ques)
    temp_sim = []
    ques_emb = sentence_embeddings[0]
    temp_ques.pop(0)
    for each in sentence_embeddings[1:]:
        similarity = cosine_similarity(ques_emb.cpu().numpy().reshape(1, -1), each.cpu().numpy().reshape(1, -1))[0][0]
        temp_sim.append(similarity)
    max_sim = max(temp_sim)
    
    question_num = []
    similarity_value = []
    for sim_i, sim in enumerate(temp_sim):
        if sim >= 0.5:
            similarity_value.append(sim)
            question_num.append(sim_i)
    if len(similarity_value) == 0:
        similarity_value.append(-1)
        question_num.append(-1)
    else:
        similarity_value, question_num = zip(*sorted(zip(similarity_value, question_num), key=lambda pair: pair[0]))
        similarity_value, question_num = list(similarity_value), list(question_num)
    
    similarity_value = ast.literal_eval(str(similarity_value))
    result = {
        'reason':reason,
        'question':question,
        'answer':answer,
        'GT_answer':GT_answer,
        'GT_reason':GT_reason,
        'generated_question':temp_ques,
        'evaluation':evaluation,
        'question_num':question_num,
        'ques_similarity_value':similarity_value,   
    }
    
    with open('../gsm8k/similarity_generate_questions/' + file_name, 'w') as f:
        json.dump(result, f, indent = 2)
    